In [ ]:
import os
import zipfile
import importlib.util
import subprocess

zip_filename = "CarDetectionDataSet.zip"
extract_folder = "archive"  # Define the folder where files are extracted

print(f"Working dir: {os.getcwd()}")  # This shows your current working directory

# Download only if the zip file and extracted folder don't exist
if not os.path.exists(zip_filename) and not os.path.exists(extract_folder):
    # Check if gdown is installed before attempting to install
    if importlib.util.find_spec("gdown") is None:
        print("gdown not found. Installing...")
        !pip install gdown

    print(f"{zip_filename} not found. Downloading...")
    !gdown 1JFAfrbUfXtiF-xwko2ACB-snDwIsj31h -O {zip_filename}
else:
    print(f"Skipping download. {zip_filename} or {extract_folder} already exists.")

# Extract only if the extracted folder does not exist
if not os.path.exists(extract_folder):
    print(f"Extracting {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"Extraction complete: {extract_folder}")
else:
    print(f"{extract_folder} already exists. Skipping extraction.")


## The data folder has been setup 

In [ ]:
if importlib.util.find_spec("ultralytics") is None:
    print("gdown not found. Installing...")
    !pip install ultralytics

if importlib.util.find_spec("torch") is None:
    print("torch not found. Installing...")
    !pip install torch

import torch
print("PyTorch Version:", torch.__version__)

In [ ]:
import os
import platform
import yaml


# Load the existing dataset.yaml configuration
with open('dataset.yaml', 'r') as f:
    dataset = yaml.safe_load(f)

# Get the current working directory
cwd = os.getcwd()

# Dynamically adjust the 'path'
# This sets an absolute path for ultralytics so that it resolves correctly
# Shouldn't be necessary but here we are
dataset['path'] = os.path.join(cwd, 'archive')

print("Resolved dataset path:", dataset['path'])

# Optionally, save this updated configuration to a new file
with open('dataset_updated.yaml', 'w') as f:
    yaml.dump(dataset, f)

In [ ]:
import pandas as pd
from ultralytics import YOLO

# Define hyperparameter grid
learning_rates = [0.002] #[0.0005, 0.002, 0.01] #, 0.005, 0.01]
epochs_list = [1] # , 20, 50] # , 15, 20, 25]

# List to store results from each experiment
results_summary = []

for lr in learning_rates:
    for epochs in epochs_list:
        print(f"Training with lr: {lr}, epochs: {epochs}")
        
        # Initialize the model
        model = YOLO("yolov8n.pt")
        
        # Train the model with the current hyperparameters
        results = model.train(
            data="dataset_updated.yaml",   # replace with your dataset YAML file path
            epochs=epochs,
            batch=8,
            imgsz=128,
            lr0=lr,
            #cache=True,
            cache='disk',
            optimizer="AdamW",
            project=f"runs/train/lr{lr}_ep{epochs}"
        )
        
        # Get the results dictionary
        rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.
        
        # Append the metrics along with the hyperparameters to our list
        results_summary.append({
            "lr": lr,
            "epochs": epochs,
            "precision": rdict.get("metrics/precision(B)", None),
            "recall": rdict.get("metrics/recall(B)", None),
            "mAP50": rdict.get("metrics/mAP50(B)", None),
            "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
            "fitness": rdict.get("fitness", None)
        })

        
        # Conditionally clear CUDA memory if available
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
        else:
            print("CUDA not available; skipping CUDA memory cleanup.")


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)

In [ ]:
import os
import yaml

# Get the current working directory
cwd = os.getcwd()

# Load dataset.yaml
with open('dataset.yaml') as f:
    dataset = yaml.safe_load(f)

# Optionally, print the resolved absolute paths for debugging:
dataset_root = os.path.join(cwd, dataset['path'])
train_path = os.path.join(dataset_root, dataset['train'])
val_path = os.path.join(dataset_root, dataset['val'])

print("Training images path:", train_path)
print("Validation images path:", val_path)
